In [2]:
import pandas as pd
import numpy as np

In [3]:
# import pertinent libraries
import os
import sys
import datetime
import glob as glob
import numpy as np
import cv2
# [Keras Models]
# import the Keras implementations of VGG16, VGG19, InceptionV3 and Xception models
# the model used here is VGG16
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import SGD
import tensorflow
# from scipy.interpolate import spline
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

2023-11-16 12:54:33.212933: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-16 12:54:33.354559: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-16 12:54:34.041154: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-16 12:54:34.041214: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-16 12:54:34.045633: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [4]:
import tensorflow as tf

In [5]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "40_classes_dataset",
    shuffle=True,
    batch_size=32,
    image_size=(299, 299)
)

labels = dataset.class_names
labels

Found 3622 files belonging to 39 classes.


2023-11-16 12:54:43.899707: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-16 12:54:43.900318: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


['Aloevera',
 'Amla',
 'Amruthaballi',
 'Arali',
 'Astma_weed',
 'Badipala',
 'Balloon_Vine',
 'Bamboo',
 'Beans',
 'Betel',
 'Bhrami',
 'Bringaraja',
 'Caricature',
 'Castor',
 'Catharanthus',
 'Chakte',
 'Chilly',
 'Citron lime (herelikai)',
 'Coffee',
 'Common rue(naagdalli)',
 'Coriender',
 'Curry',
 'Doddpathre',
 'Drumstick',
 'Ekka',
 'Eucalyptus',
 'Ganigale',
 'Ganike',
 'Gasagase',
 'Ginger',
 'Globe Amarnath',
 'Guava',
 'Henna',
 'Hibiscus',
 'Honge',
 'Insulin',
 'Jackfruit',
 'ashoka',
 'camphor']

In [6]:
import numpy as np

for image_batch, labels_batch in dataset.take(1):
    print(image_batch.shape)
    print(labels_batch.numpy())
    break

(32, 299, 299, 3)
[22 21 30 14 28 22  6 36 29 28  2 38 36 33 14 13  6 26 23 12  5  3 24  6
 35 35 15 36 21 15  6  9]


In [7]:
# train test split
train_size = int(0.8 * len(dataset))
test_size = int(0.2 * len(dataset))
train_size, test_size

(91, 22)

In [8]:
def get_dataset_partisions_tf(ds, train_split=0.8, test_split=0.2, shuffle=True, shuffle_size=10000):
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    train_size = int(train_split * len(ds))
    test_size = int(test_split * len(ds))
    train_ds = ds.take(train_size)
    test_ds = ds.skip(train_size)
    val_ds = test_ds.skip(test_size)
    test_ds = test_ds.take(test_size)
    return train_ds, test_ds, val_ds

In [9]:
train_ds, test_ds, val_ds = get_dataset_partisions_tf(dataset)
len(train_ds), len(test_ds), len(val_ds)

(91, 22, 1)

In [10]:
resize_and_rescale = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Resizing(299, 299),
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
])

In [11]:
# train using InceptionV3
base_model = tf.keras.applications.InceptionV3(
    weights='imagenet',
    input_shape=(299, 299, 3),
    include_top=False,
    pooling='avg',
    classifier_activation='softmax',
    classes=len(labels)
)

In [12]:
base_model.trainable = False

inputs = tf.keras.Input(shape=(299, 299, 3))
x = resize_and_rescale(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(len(labels), activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

history = model.fit(
    train_ds,
    validation_data=val_ds,
    batch_size=32,
    epochs=20
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 299, 299, 3)       0         
                                                                 
 inception_v3 (Functional)   (None, 2048)              21802784  
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 39)                5031      
                                                                 
Total params: 22070087 (84.19 MB)
Trainable params: 267303 (1

2023-11-16 12:55:03.457397: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:9: Filling up shuffle buffer (this may take a while): 68 of 10000
2023-11-16 12:55:08.185513: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


91/91 [==============================] - ETA: 0s - loss: 3.0124 - accuracy: 0.2222

2023-11-16 12:57:43.960161: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:9: Filling up shuffle buffer (this may take a while): 77 of 10000
2023-11-16 12:57:46.594852: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


91/91 [==============================] - 178s 2s/step - loss: 3.0124 - accuracy: 0.2222 - val_loss: 2.0935 - val_accuracy: 0.5000
Epoch 2/20


...

91/91 [==============================] - ETA: 0s - loss: 0.2999 - accuracy: 0.9269

2023-11-16 13:32:27.068677: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:9: Filling up shuffle buffer (this may take a while): 99 of 10000
2023-11-16 13:32:27.727077: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


91/91 [==============================] - 141s 1s/step - loss: 0.2999 - accuracy: 0.9269 - val_loss: 0.1976 - val_accuracy: 0.9062
Epoch 15/20


2023-11-16 13:32:40.025807: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:9: Filling up shuffle buffer (this may take a while): 104 of 10000
2023-11-16 13:32:40.123416: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


91/91 [==============================] - 142s 1s/step - loss: 0.2727 - accuracy: 0.9293 - val_loss: 0.0896 - val_accuracy: 1.0000
Epoch 16/20
91/91 [==============================] - 141s 1s/step - loss: 0.2498 - accuracy: 0.9380 - val_loss: 0.0726 - val_accuracy: 1.0000
Epoch 17/20
91/91 [==============================] - 141s 1s/step - loss: 0.2226 - accuracy: 0.9477 - val_loss: 0.0524 - val_accuracy: 1.0000
Epoch 18/20
91/91 [==============================] - 141s 1s/step - loss: 0.2247 - accuracy: 0.9439 - val_loss: 0.1234 - val_accuracy: 1.0000
Epoch 19/20
91/91 [==============================] - 142s 1s/step - loss: 0.2094 - accuracy: 0.9463 - val_loss: 0.0305 - val_accuracy: 1.0000
Epoch 20/20
91/91 [==============================] - 142s 1s/step - loss: 0.1876 - accuracy: 0.9563 - val_loss: 0.0862 - val_accuracy: 0.9688


In [16]:
# predict with new images
import numpy as np

img = tf.keras.preprocessing.image.load_img(
    'test_images/aloeleaf.jpg', target_size=(299, 299)
)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create a batch


predictions = model.predict(img_array)
score = tf.nn.sigmoid(predictions[0])
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(labels[np.argmax(score)], 100 * np.max(score))
)

1/1 [==============================] - 0s 70ms/step
This image most likely belongs to Aloevera with a 69.26 percent confidence.


In [24]:
# predict with new images
import numpy as np

img = tf.keras.preprocessing.image.load_img(
    'test_images/ashoka.jpg', target_size=(299, 299)
)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create a batch


predictions = model.predict(img_array)
score = tf.nn.sigmoid(predictions[0])
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(labels[np.argmax(score)], 100 * np.max(score))
)

1/1 [==============================] - 0s 156ms/step
This image most likely belongs to ashoka with a 71.85 percent confidence.


In [ ]:
# predict with new images
import numpy as np

img = tf.keras.preprocessing.image.load_img(
    'test_images/images.jpg', target_size=(299, 299)
)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create a batch


predictions = model.predict(img_array)
score = tf.nn.sigmoid(predictions[0])
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(labels[np.argmax(score)], 100 * np.max(score))
)

In [30]:
model.save('model_40_classes_inception.h5')

/home/fash1462/.local/bin/.virtualenvs/be_proj/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
